In [ ]:
import pandas as pd
import json
import requests
import jsonlines
import csv
import random
import torch
import time
from tqdm import tqdm
import numpy as np

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

In [ ]:
device = "cuda:1" # the device to load the model onto

model_id = "/users/wangdongnian/outputs/ckpt/qwen2_7b_xf_v3-20240705-195808/iter_0000429/huggingface_format"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map=device,
    trust_remote_code=True
)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
# lora_path = "/mnt/dolphinfs/hdd_pool/docker/user/hadoop-mtai-llms/users/wangdongnian/outputs/ckpt/qwen2_7b_xf_lora_r64_a16_epoch5/last_ft_model"
# model = PeftModel.from_pretrained(model, model_id=lora_path)

In [ ]:
def qwen_infer(dialogue, model, tokenizer, prompt_text=prompt_exect):
    content = prompt_text.replace("##Content##", dialogue)
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": content}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # try:
    #     if '```json' in response:
    #         response = response.replace('```json', '').replace('```', '')
    #     # response = json.loads(response)
    # except json.JSONDecodeError as e:
    #     print("JSONDecodeError : " + response)
    #     raise e

    return response

In [ ]:
def convert_all_json_in_text_to_dict(text):
    """Extracts JSON objects from the text."""
    dicts, stack = [], []
    for i in range(len(text)):
        if text[i] == '{':
            stack.append(i)
        elif text[i] == '}':
            begin = stack.pop()
            if not stack:
                dicts.append(json.loads(text[begin:i+1]))
    return dicts

answer = qwen_infer(test_data.iloc[0]["chat_text"], model, tokenizer, prompt_exect)
print(answer)
infos = convert_all_json_in_text_to_dict(answer.replace("\'", "\""))
# infos = json.load(answer)
print(infos)